Preparing necessary imports

In [10]:
from ipywidgets import widgets, interact, fixed
from IPython.display import display
from getpass import getpass
import omero.gateway as gw
import omero

from plots import plot_homogeneity_map, plot_distances_map, plot_psfs, plot_distances_map_bokeh
from utils import get_tables

try:
    from credentials import *
except ModuleNotFoundError:
    HOST = input('host:')
    PORT = 4064
    USER = input('user:')
    PASSWORD = getpass('password')
    GROUP = input('group:')

### Defining display functions and a tag_to_function mapping

In [11]:
def print_map_ann(image):
    print(f'Measurements for image id: {image.getId()}\n{image.getName()}')
    for ann in image.listAnnotations():
        if ann.OMERO_TYPE == omero.model.MapAnnotationI and \
            ann.getNs().startswith('metrics'):
            print('Key\tvalue')
            for m in ann.getValue():
                print(f'{m[0]}\t{m[1]}')

def display_argoSIM_E(image):
    print_map_ann(image)
    print('Some profiles should go in here')

def display_argoSIM_B(image):
    print_map_ann(image)
    plot_homogeneity_map(image)
    plot_distances_map(image)
    plot_distances_map_bokeh(image)

def display_psfs(image):
    properties_tables = get_tables(image, namespace_start='metrics', name_filter='properties')
    if len(properties_tables) != 1:
        raise Exception('There are none or more than one distances tables. Verify data integrity.')
    properties_table = properties_tables[0]
    bead_count = properties_table.getNumberOfRows()
    print(f'There are {bead_count} beads associated with image {image.getName()}. Choose which bead bead you want to see, 0 for none or all.')
    options = [0]
    options.extend([b+1 for b in range(bead_count)])
    options.append('all')
    interact(plot_psfs, image=fixed(image), bead_nr=options)


tag_to_function = {'Argo-SIM_E_90deg': display_argoSIM_E,
                   'Argo-SIM_E_0deg': display_argoSIM_E,
                   'Argo-SIM_B': display_argoSIM_B,
                   'PSF beads': display_psfs}

### Open connection

In [12]:
conn = gw.BlitzGateway(username=USER,
                       passwd=PASSWORD,
                       host=HOST,
                       group=GROUP,
                       port=PORT)

# conn = gw.BlitzGateway(username=input('OMERO username:'), password=getpass('OMERO password:'),
#                        group=GROUP, host=OMERO_SERVER, port=PORT, secure=True)

conn.connect()
print(f'Connection successful: {conn.isConnected()}')

Connection successful: True


### Selecting the Microscope

In [13]:
microscopes = list(conn.getObjects('Project'))
micr_menu = widgets.Dropdown(options=[(m.getName(), i) for i, m in enumerate(microscopes)],
                     value=1,
                     description='Select a microscope:',
                     disabled=False,)
display(micr_menu)

Dropdown(description='Select a microscope:', index=1, options=(('microscope_2', 0), ('microscope_1', 1)), valu…

### Selecting the Dataset

In [14]:
microscope = microscopes[micr_menu.value]

datasets = list(microscope.listChildren())
dataset_menu = widgets.Dropdown(options=[(d.getName(), i) for i, d in enumerate(datasets)],
                     value=0,
                     description='Select a dataset:',
                     disabled=False,)
display(dataset_menu)

Dropdown(description='Select a dataset:', options=(('2017-02-20_yearly', 0), ('2017-03-20_monthly', 1), ('2017…

### Select the type of analysis to display

In [15]:
dataset = datasets[dataset_menu.value]
images = list(dataset.listChildren())

# Get the relevant tags associated to the images in the dataset
relevant_tags = ['Argo-SIM_B', 'AArgo-SIM_E_0deg', 'Argo-SIM_E_90deg', 'PSF beads']
present_tags = []

for image in images:
    for ann in image.listAnnotations():
        if ann.OMERO_TYPE == omero.model.TagAnnotationI and \
                ann.getTextValue() in relevant_tags and \
                ann.getTextValue() not in present_tags:
            present_tags.append(ann.getTextValue())

tags_menu = widgets.Dropdown(options=[tag for tag in present_tags],
                             description='Select an analysis type:',
                             value='PSF beads',
                             disabled=False,)
display(tags_menu)

Dropdown(description='Select an analysis type:', index=2, options=('Argo-SIM_E_90deg', 'Argo-SIM_B', 'PSF bead…

### Run visualization
Select an image to visualize

In [17]:
images_to_visualize = list()
for image in images:
    for ann in image.listAnnotations():
        if ann.OMERO_TYPE == omero.model.TagAnnotationI and \
                ann.getTextValue() == tags_menu.value:
            images_to_visualize.append(image)

image_names = [(i.getName(), i) for i in images_to_visualize]

# if tags_menu == 'PSF beads':
interact(tag_to_function[tags_menu.value], image=image_names)


            # if tags_menu.value == 'PSF beads':
            #     interact(display_psfs, image=fixed(image))
            # tag_to_function[tags_menu.value](image)

interactive(children=(Dropdown(description='image', options=(('201702_RI510_Argolight-1-1_004_SIR_ALX_THR.ome.…

<function __main__.display_argoSIM_E(image)>